<a href="https://colab.research.google.com/github/kimsooyoung/iap_hanyang/blob/main/tutorial06_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        # LeNet이라 불리는 모델임
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # <img width="613" alt="image" src="https://user-images.githubusercontent.com/12381733/160575164-84dad783-1c86-4ad5-8826-49125c41ee1c.png">
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

for i in range(len(params)):
  print(params[i].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [ ]:
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output)

tensor([[ 0.1337, -0.0800,  0.0863, -0.0080, -0.1371, -0.0411, -0.0605,  0.1767,
         -0.0350, -0.0802]], grad_fn=<AddmmBackward0>)


In [ ]:
torch.randn(10).view(1, -1)

tensor([[ 0.3440,  0.7985, -1.1417, -1.5049, -0.5218, -0.4713,  0.0683,  0.2591,
         -1.6919,  1.5151]])

In [ ]:
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output => 1차원 더 패딩을 하였음
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7421, grad_fn=<MseLossBackward0>)


In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0047, -0.0102, -0.0016, -0.0093,  0.0101,  0.0081])


In [ ]:
x = net.conv1.bias.detach()
x_grad = net.conv1.bias.grad.detach()
print('Before weight update: ', x.data)
print('Gradients for weight: ', x_grad.data)

# Stochastic gradient descent: w = w - lr * wgrad
learning_rate = 0.01
x_new = x - (x_grad * learning_rate)
print('After weight update : ', x_new.data)

# learning_rate = 0.01
# x.data.sub_(x.grad.data * learning_rate)

Before weight update:  tensor([ 0.3162, -0.1054, -0.1281,  0.2160,  0.1284, -0.0236])
Gradients for weight:  tensor([ 0.0047, -0.0102, -0.0016, -0.0093,  0.0101,  0.0081])
After weight update :  tensor([ 0.3161, -0.1053, -0.1281,  0.2160,  0.1283, -0.0237])


In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

print('After optim step : ', net.conv1.bias.data)

After optim step :  tensor([ 0.3161, -0.1053, -0.1281,  0.2160,  0.1283, -0.0237])


## Save/load models

In [11]:
stored_obj = {
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(stored_obj, 'fileName.pt')

In [12]:
new_model = Net()
new_optimizer = optim.SGD(new_model.parameters(), lr=0.01)

checkpoint = torch.load('fileName.pt')
new_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])